## AIM

In this notebook, I checked the accuracy of the Google Scholar dataset. I found that most of them are pretty accurate. Only 124 of them are not accurate. 

In [1]:
import pandas as pd
import numpy as np
import re
from difflib import SequenceMatcher
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException 
from selenium.common.exceptions import ElementNotInteractableException
import os
import random
import re
import csv
import urllib.parse

In [2]:
df = pd.read_csv('../../data/processed/gscholar_data.csv')
df.head()

,Original Title,Title on Google Scholar,DOI,Year,Citation Link,Citation Counts on Google Scholar
0,The Gender Divide in Wikipedia: Quantifying an...,The Gender Divide in Wikipedia: Quantifying an...,10.1093/joc/jqac004,2022,https://scholar.google.com/scholar?cites=73284...,2
1,Mapping Exposure Diversity: The Divergent Effe...,Mapping exposure diversity: The divergent effe...,10.1093/joc/jqac009,2022,https://scholar.google.com/scholar?cites=29888...,15
2,Democratic Consequences of Incidental Exposure...,Democratic consequences of incidental exposure...,10.1093/joc/jqac008,2022,https://scholar.google.com/scholar?cites=13540...,13
3,The Great and Powerful Dr. Oz? Alternative Hea...,The great and powerful Dr. Oz? Alternative hea...,10.1093/joc/jqac011,2022,https://scholar.google.com/scholar?cites=12065...,7
4,A Methodological Framework for Analyzing the A...,A methodological framework for analyzing the a...,10.1093/joc/jqac013,2022,https://scholar.google.com/scholar?cites=31605...,1


In [3]:
df.shape

(5718, 6)

In [4]:
df.shape

(5718, 6)

In [5]:
len(list(set(df.DOI.tolist())))

5717

### Which paper appeared for twice

In [9]:
from collections import Counter

In [12]:
Counter(df.DOI).most_common(2)

[('10.1111/j.1083-6101.1996.tb00053.x', 2), ('10.1093/joc/jqac004', 1)]

In [13]:
df.loc[df.DOI == '10.1111/j.1083-6101.1996.tb00053.x']

,Original Title,Title on Google Scholar,DOI,Year,Citation Link,Citation Counts on Google Scholar
5220,Cyberspaces: Familiar Territory or Lawless Fro...,Cyberspaces: Familiar territory or lawless fro...,10.1111/j.1083-6101.1996.tb00053.x,1996,https://scholar.google.com/scholar?cites=33144...,24
5229,Cyberspaces: Familiar Territory or Lawless Fro...,Cyberspaces: Familiar territory or lawless fro...,10.1111/j.1083-6101.1996.tb00053.x,1996,https://scholar.google.com/scholar?cites=33144...,24


### One paper is not found

In [78]:
all_dois = pd.read_csv('../../data/processed/dois_to_study.txt', header=None)
all_dois_list = all_dois.iloc[:, 0].tolist()

not_included_doi = [x for x in all_dois_list if x not in df['DOI'].tolist()]

In [80]:
not_included_doi

['10.1111/j.1083-6101.1996.tb00178.x']

In [95]:
list(set(all_dois_list).difference(df['DOI'].tolist()))

['10.1111/j.1083-6101.1996.tb00178.x']

In [85]:
not_included_doi[0]

'10.1111/j.1083-6101.1996.tb00178.x'

In [34]:
papers_to_study = pd.read_csv('../../data/processed/papers_to_study.csv')
papers_to_study.loc[papers_to_study['doi'] == not_included_doi[0]]

,journal,issueURL,volumn,issue,issueText,month,year,category,title,url,doi,pages,abstract,abstract_para_num,type
5229,Journal of Computer-Mediated Communication,https://academic.oup.com/jcmc/issue/2/1?browse...,Volume 2,Issue 1,"Issue 1, 1 June 1996",June,1996,Original Article,Cyberspaces: Familiar Territory or Lawless Fro...,https://academic.oup.com/jcmc/article/2/1/JCMC...,10.1111/j.1083-6101.1996.tb00178.x,NaN,NaN,NaN,R


In [43]:
papers_to_study.loc[papers_to_study.doi == '10.1111/j.1460-2466.1952.tb00171.x']

,journal,issueURL,volumn,issue,issueText,month,year,category,title,url,doi,pages,abstract,abstract_para_num,type
2595,Journal of Communication,https://academic.oup.com/joc/issue/2/1?browseB...,Volume 2,Issue 1,"Issue 1, May 1952, Pages 1–98",May,1952,Articles,Communications 1952,https://academic.oup.com/joc/article/2/1/11/46...,10.1111/j.1460-2466.1952.tb00171.x,11–14,NaN,NaN,R


In [50]:
papers_to_study.loc[papers_to_study.doi == '10.1111/j.1460-2466.1977.tb02133.x']
## This paper is not available on google scholar. 

,journal,issueURL,volumn,issue,issueText,month,year,category,title,url,doi,pages,abstract,abstract_para_num,type
1946,Journal of Communication,https://academic.oup.com/joc/issue/27/3?browse...,Volume 27,Issue 3,"Issue 3, September 1977, Pages 3–240",September,1977,Articles,“Beneficial” Aggression,https://academic.oup.com/joc/article/27/3/100/...,10.1111/j.1460-2466.1977.tb02133.x,100–103,NaN,NaN,R


In [51]:
papers_to_study.loc[papers_to_study.doi == '10.1111/j.1083-6101.2008.01429.x']

,journal,issueURL,volumn,issue,issueText,month,year,category,title,url,doi,pages,abstract,abstract_para_num,type
4811,Journal of Computer-Mediated Communication,https://academic.oup.com/jcmc/issue/14/1?brows...,Volume 14,Issue 1,"Issue 1, 1 October 2008, Pages 1–220",October,2008,Original Article,The Faces of Facebookers: Investigating Social...,https://academic.oup.com/jcmc/article/14/1/1/4...,10.1111/j.1083-6101.2008.01429.x,NaN,This research investigates two competing hypot...,1.0,R


This is very confusing to me. I do not know why this paper is ignored by my script. So confusing!

## Export

In [118]:
with open("gscholar_data_dois.txt", "w") as output:
    for row in df.DOI.tolist():
        output.write(str(row) + '\n')

In [117]:
with open("all_dois_list.txt", "w") as output:
    for row in all_dois_list:
        output.write(str(row) + '\n')

## Compare 

In [4]:
def clean_text(text):
    """
    Takes a string and returns a string
    """
    # remove html tags, lowercase, remove nonsense, remove non-letter
    title = text.replace('[CITATION]', '')
    title = title.replace('[PDF]', '')
    title = title.lower()
    title = re.sub(r'[^a-z]+', ' ', title)
    title = title.strip()
    return title

In [5]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [6]:
def get_similar_score(row):
    orig_t = row['orig_t_processed']
    gscholar_t = row['gscholar_t_processed']
    return similar(orig_t, gscholar_t)

In [7]:
df['orig_t_processed'] = [clean_text(i) for i in df['Original Title']]
df['gscholar_t_processed'] = [clean_text(i) for i in df['Title on Google Scholar']]

In [8]:
dff = df[df['orig_t_processed'] != df['gscholar_t_processed']]

In [9]:
dff.head(3)

,Original Title,Title on Google Scholar,DOI,Year,Citation Link,Citation Counts on Google Scholar,orig_t_processed,gscholar_t_processed
7,"#GirlGamers, Soldiers, and Public Relations: A...",User profiles for good,10.1093/joc/jqab051,2022,https://scholar.google.com/scholar?cites=86484...,864,girlgamers soldiers and public relations analy...,user profiles for good
17,What Does #Freedom Look Like? Instagram and th...,How does foreign direct investment affect econ...,10.1093/joc/jqab021,2021,https://scholar.google.com/scholar?cites=17920...,10630,what does freedom look like instagram and the ...,how does foreign direct investment affect econ...
59,Intellectuals Debate #MeToo in China: Legitimi...,[HTML] How to build a 'Beautiful China'in the ...,10.1093/joc/jqaa033,2020,https://scholar.google.com/scholar?cites=13177...,121,intellectuals debate metoo in china legitimizi...,html how to build a beautiful china in the ant...


In [10]:
dff.shape

(289, 8)

In [35]:
dff = dff.assign(simi_score = dff.apply(lambda row: get_similar_score(row), axis = 1))

In [36]:
dff[dff.simi_score <= 0.95].head()

,Original Title,Title on Google Scholar,DOI,Year,Citation Link,Citation Counts on Google Scholar,orig_t_processed,gscholar_t_processed,simi_score
7,"#GirlGamers, Soldiers, and Public Relations: A...",User profiles for good,10.1093/joc/jqab051,2022,https://scholar.google.com/scholar?cites=86484...,864,girlgamers soldiers and public relations analy...,user profiles for good,0.192982
17,What Does #Freedom Look Like? Instagram and th...,How does foreign direct investment affect econ...,10.1093/joc/jqab021,2021,https://scholar.google.com/scholar?cites=17920...,10630,what does freedom look like instagram and the ...,how does foreign direct investment affect econ...,0.320611
59,Intellectuals Debate #MeToo in China: Legitimi...,[HTML] How to build a 'Beautiful China'in the ...,10.1093/joc/jqaa033,2020,https://scholar.google.com/scholar?cites=13177...,121,intellectuals debate metoo in china legitimizi...,html how to build a beautiful china in the ant...,0.270916
71,"Past Debates, Fresh Impact on Nano-Enabled Foo...","Past debates, fresh impact on nano-enabled foo...",10.1093/joc/jqaa019,2020,https://scholar.google.com/scholar?cites=13485...,16,past debates fresh impact on nano enabled food...,past debates fresh impact on nano enabled food...,0.920245
90,"Toward an Aggregate, Implicit, and Dynamic Mod...","Toward an aggregate, implicit, and dynamic mod...",10.1093/joc/jqz033,2019,https://scholar.google.com/scholar?cites=26600...,16,toward an aggregate implicit and dynamic model...,toward an aggregate implicit and dynamic model...,0.901163


In [37]:
dff[dff.simi_score <= 0.95].shape

(123, 9)

In [38]:
# these are the 123 papers to scrape again
to_scrape_further = dff[dff.simi_score <= 0.95]

## Scrape these papers

In [46]:
papers = to_scrape_further['Original Title'].tolist()
dois = to_scrape_further['DOI'].tolist()
len(papers), papers[0], dois[0]

(123,
 '#GirlGamers, Soldiers, and Public Relations: Analyzing Gender Representation in U.S. Army Esports',
 '10.1093/joc/jqab051')

In [47]:
gscholar_dict_list = []
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 90)

In [48]:
for paper in papers:
    idx = papers.index(paper)
    doi = dois[idx]
    # I decided to remove this paper as it is not available on gscholar
    if doi != '10.1111/j.1460-2466.1952.tb00171.x':
        query_string = 'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C50&q='
        driver.get(query_string + doi + '&btnG=')
        gs_paper_e = wait.until(EC.presence_of_element_located((
                By.CSS_SELECTOR, 'h3.gs_rt')))
        gs_paper_title = gs_paper_e.text
        gs_citation_e = wait.until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="gs_fl"]//child::a[3]'
        )))
        citation_link = gs_citation_e.get_attribute('href')
        citation_count_string = gs_citation_e.get_attribute('innerHTML')
        if 'Cited by' in citation_count_string:
            gs_citation_count = int(re.findall(r'\d+', citation_count_string)[0])
        else:
            gs_citation_count = 0
        gscholar_dict = {
            'Original Title': paper,
            'Title on Google Scholar': gs_paper_title,
            'DOI': doi,
            'Citation Link': citation_link,
            'Citation Counts on Google Scholar': gs_citation_count,
        }
        gscholar_dict_list.append(gscholar_dict)
        print(f'paper {idx + 1} is done!')
        time.sleep(0.2+random.uniform(0,0.2))

paper 1 is done!
paper 2 is done!
paper 3 is done!
paper 4 is done!
paper 5 is done!
paper 6 is done!
paper 7 is done!
paper 8 is done!
paper 9 is done!
paper 10 is done!
paper 11 is done!
paper 12 is done!
paper 13 is done!
paper 14 is done!
paper 15 is done!
paper 16 is done!
paper 17 is done!
paper 18 is done!
paper 19 is done!
paper 20 is done!
paper 21 is done!
paper 22 is done!
paper 23 is done!
paper 24 is done!
paper 25 is done!
paper 26 is done!
paper 27 is done!
paper 28 is done!
paper 29 is done!
paper 30 is done!
paper 31 is done!
paper 32 is done!
paper 33 is done!
paper 34 is done!
paper 35 is done!
paper 36 is done!
paper 37 is done!
paper 38 is done!
paper 39 is done!
paper 40 is done!
paper 41 is done!
paper 42 is done!
paper 43 is done!
paper 44 is done!
paper 45 is done!
paper 46 is done!
paper 47 is done!
paper 48 is done!
paper 49 is done!
paper 50 is done!
paper 51 is done!
paper 52 is done!
paper 53 is done!
paper 54 is done!
paper 55 is done!
paper 56 is done!
p

In [49]:
gscholar_data_manual = pd.DataFrame(gscholar_dict_list)
gscholar_data_manual.to_csv('../../data/interim/gscholar_data_manual.csv', index = False)